In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
import math 
import xarray as xr

In [ ]:
filedir = '/uio/lagringshotell/geofag/students/metos/kaervik/Master/'

uvar = xr.open_mfdataset(filedir + 'uvar.nc')
vvar = xr.open_mfdataset(filedir + 'vvar.nc')
uv = xr.open_mfdataset(filedir + 'uv.nc')
uv = uv.rename({'__xarray_dataarray_variable__' : 'vel'})
bat = xr.open_mfdataset(filedir + '/CREG12.L75.script/bathym.nc')


In [ ]:
# Getting the bottom velocity 
d = bat.mbathy.values-1
d[d==-1] = 0

d = xr.DataArray(d, dims=['y','x'])

uvar_b = uvar.isel(depth=d)
vvar_b = vvar.isel(depth=d)
uv_b = uv.isel(depth=d)

In [ ]:
# Smoothening bottom velocity data 
uvar_b = uvar_b.rolling(x=10).mean()
vvar_b = vvar_b.rolling(y=10).mean()
uv_b = uv_b.rolling(x=10).mean()
uv_b = uv_b.rolling(y=10).mean()

print(uvar_b)
print('------------')
print(uv_b)
print('-------------')
#le = la 


In [ ]:
# List of x,y elements
dx = 20
dy = 20
#dx = 10 # need to figure out what the PC list are gonna be when dx dy = 10 
#dy = 10

jmax, imax = uv_b.vel.shape
i_list = np.arange(0,imax,dx)
j_list = np.arange(0,jmax,dy)

In [ ]:
#'''
# Making the covmatrix and finding the eigenvalues and eigenvectors
# ----------------------------------------------------------------------
PC11 = np.zeros(3379)
PC12 = np.zeros(3379)
PC21 = np.zeros(3379)
PC22 = np.zeros(3379)
y = np.zeros(3379)
x = np.zeros(3379)
indx = 0

In [ ]:
# to get the loop to run faster
uvy = np.array(uv.y.values)
uvx = np.array(uv.x.values)

# surface values
uvarb = np.array(uvar_b.vozocrtx.values)
vvarb = np.array(vvar_b.vomecrty.values)
uvvel = np.array(uv_b.vel.values)

In [ ]:
for i in i_list:
    for j in j_list:
        covm = np.array([[uvarb[j,i],uvvel[j,i]],
        [uvvel[j,i],vvarb[j,i]]],dtype=float)
        if np.isnan(covm).any():
            continue
        else:
            values, vectors = np.linalg.eig(covm)
            PC11[int(indx)] = vectors[0,0]*np.sqrt(values[0])
            PC12[int(indx)] = vectors[1,0]*np.sqrt(values[0])
            PC21[int(indx)] = vectors[0,1]*np.sqrt(values[1])
            PC22[int(indx)] = vectors[1,1]*np.sqrt(values[1])
            y[int(indx)] = uvy[j]
            x[int(indx)] = uvx[i]
            indx = indx + 1



In [ ]:
# converting to np.arrays
PC11 = np.array(PC11,dtype=float)
PC12 = np.array(PC12,dtype=float)
PC21 = np.array(PC21,dtype=float)
PC22 = np.array(PC22,dtype=float)
y = np.array(y,dtype=float)
x = np.array(x,dtype=float)

In [ ]:
# Plot
fig, ax = plt.subplots(1,1,figsize=(10,10))
cmap = plt.get_cmap('magma_r')
ax.contour(bat.mbathy, colors='grey',levels=8, linewidths=0.4)
ax.quiver( x[::1], y[::1], PC11[::1], PC12[::1], 
    #color='r',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=4)
ax.quiver(x[::1], y[::1], -PC11[::1], -PC12[::1],
    #color='r',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=4)
ax.quiver( x[::1], y[::1], PC21[::1], PC22[::1],
    #color='b',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=4)
ax.quiver( x[::1], y[::1], -PC21[::1], -PC22[::1],
    #color='b',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=4)
ax.set_xlim(200,1400)
ax.set_ylim(800,1800)
plt.title('PC plot of the bottom current')

plt.show()